In [1]:
!pip install bing-image-downloader

In [7]:
!mkdir Input_images

from bing_image_downloader import downloader

mkdir: cannot create directory ‘Input_images’: File exists


In [6]:
characters = [
    "Iron Man", "Spider-Man", "Captain America", "Hulk", "Thor",
    "Black Widow", "Doctor Strange", "Black Panther", "Captain Marvel", "Ant-Man",
    "Naruto", "Goku", "Luffy", "Saitama", "Deku",
    "Sasuke", "Vegeta", "Ichigo", "Natsu", "Eren"
]

In [8]:
# Function to download images for each character
def download_images(character_list, limit=30):
    for character in character_list:
        print(f"Downloading images for {character}...")
        downloader.download(character + " figurine", limit=limit, output_dir='Input_images/Figurines', adult_filter_off=True)
        print(f"Downloaded images for {character}")

In [9]:
# Download images for all characters
download_images(characters)

[%] Downloading Images to /content/Input_images/Figurines/Iron Man figurine


[!!]Indexing page: 1

[%] Indexed 30 Images on Page 1.


[%] Downloading Image #1 from https://cdn11.bigcommerce.com/s-0kvv9/images/stencil/2560w/products/147502/205223/apinmy2mr__60684.1516123244.jpg?c=2
[%] File Downloaded !

[%] Downloading Image #2 from https://m.media-amazon.com/images/I/71KRXFSGGQL._AC_SL1500_.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://www.reference-gaming.com/assets/media/product/41653/figurine-sh-figuarts-avengers-iron-man-5aa110062cbd3.jpg?format=product-cover-large&amp;k=1520504811
[%] File Downloaded !

[%] Downloading Image #4 from https://www.pro-bems.com/IMAGES/images_1/FIGHOT909463/m/FIGHOT909463_2.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://actionfiguresdaily.com/wp-content/uploads/2018/01/Mezco-Iron-Man-One12-Collective-008.jpg
[%] File Downloaded !

[%] Downloading Image #6 from http://marveltoynews.com/wp-content/uploads/2013/04/Ha

Our dataset is downloaded now for 20 characters with 30 images for each character making it to function well.

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


In [11]:
# Path to the dataset
data_dir = 'Input_images/Figurines'

In [12]:
# Create ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,       # Normalize pixel values to [0,1]
    shear_range=0.2,      # Randomly apply shearing
    zoom_range=0.2,       # Randomly zoom into images
    horizontal_flip=True, # Randomly flip images horizontally
    validation_split=0.2  # Split for validation
)


In [13]:
# Create training data generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=32,           # Number of images to process in a batch
    class_mode='categorical',# Use categorical labels
    subset='training'        # Specify this is the training subset
)

Found 480 images belonging to 20 classes.


In [14]:

# Create validation data generator
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=32,           # Number of images to process in a batch
    class_mode='categorical',# Use categorical labels
    subset='validation'      # Specify this is the validation subset
)

Found 120 images belonging to 20 classes.


Now lets create our CNN model

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [16]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),  # First convolutional layer
    MaxPooling2D(pool_size=(2, 2)),                                    # First max pooling layer
    Conv2D(64, (3, 3), activation='relu'),                             # Second convolutional layer
    MaxPooling2D(pool_size=(2, 2)),                                    # Second max pooling layer
    Conv2D(128, (3, 3), activation='relu'),                            # Third convolutional layer
    MaxPooling2D(pool_size=(2, 2)),                                    # Third max pooling layer
    Flatten(),                                                         # Flatten the 3D output to 1D
    Dense(512, activation='relu'),                                     # Fully connected layer with 512 units
    Dropout(0.5),                                                      # Dropout for regularization
    Dense(len(train_generator.class_indices), activation='softmax')    # Output layer with number of classes
])

In [18]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=25
)

Epoch 1/25
 7/15 [=============>................] - ETA: 21s - loss: 3.9560 - accuracy: 0.0312

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


15/15 [==============================] - 48s 3s/step - loss: 3.4493 - accuracy: 0.0312 - val_loss: 2.9916 - val_accuracy: 0.0521
Epoch 2/25
15/15 [==============================] - 50s 3s/step - loss: 2.9797 - accuracy: 0.0688 - val_loss: 2.9576 - val_accuracy: 0.1771
Epoch 3/25
15/15 [==============================] - 48s 3s/step - loss: 2.8653 - accuracy: 0.1312 - val_loss: 2.7726 - val_accuracy: 0.2188
Epoch 4/25
15/15 [==============================] - 49s 3s/step - loss: 2.6143 - accuracy: 0.2354 - val_loss: 2.5107 - val_accuracy: 0.2708
Epoch 5/25
15/15 [==============================] - 46s 3s/step - loss: 2.3557 - accuracy: 0.2958 - val_loss: 2.2855 - val_accuracy: 0.2812
Epoch 6/25
15/15 [==============================] - 45s 3s/step - loss: 2.0966 - accuracy: 0.3708 - val_loss: 2.2020 - val_accuracy: 0.3750
Epoch 7/25
15/15 [==============================] - 49s 3s/step - loss: 2.0587 - accuracy: 0.3854 - val_loss: 2.0757 - val_accuracy: 0.3542
Epoch 8/25
15/15 [=============

In [20]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

# Save the model
model.save('figurine_classifier.h5')


4/4 [==============================] - 6s 1s/step - loss: 2.0704 - accuracy: 0.4917
Validation Accuracy: 49.17%


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
# Function to predict the character from a new image
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    class_labels = list(train_generator.class_indices.keys())
    return class_labels[class_idx]



In [37]:
!pip install wget

In [34]:
import wget
image_url = "https://ichef.bbci.co.uk/images/ic/1200x675/p09t1hg0.jpg"
image_filename = "my_favourite.jpg"
wget.download(image_url, image_filename)

'my_favourite (3).jpg'

In [35]:

# Test the prediction function
image_path = "my_favourite.jpg"
print(f'The given image is: {predict_image(image_path)}')


1/1 [==============================] - 0s 61ms/step
The given image is: Captain Marvel figurine


In [39]:

# Print the class indices to verify correct labeling
print("Class indices:", train_generator.class_indices)

# Evaluate the model and print the accuracy
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

# Check the class indices
class_labels = list(train_generator.class_indices.keys())
print(f'Class labels: {class_labels}')

Class indices: {'Ant-Man figurine': 0, 'Black Panther figurine': 1, 'Black Widow figurine': 2, 'Captain America figurine': 3, 'Captain Marvel figurine': 4, 'Deku figurine': 5, 'Doctor Strange figurine': 6, 'Eren figurine': 7, 'Goku figurine': 8, 'Hulk figurine': 9, 'Ichigo figurine': 10, 'Iron Man figurine': 11, 'Luffy figurine': 12, 'Naruto figurine': 13, 'Natsu figurine': 14, 'Saitama figurine': 15, 'Sasuke figurine': 16, 'Spider-Man figurine': 17, 'Thor figurine': 18, 'Vegeta figurine': 19}
4/4 [==============================] - 6s 1s/step - loss: 1.9837 - accuracy: 0.5167
Validation Accuracy: 51.67%
Class labels: ['Ant-Man figurine', 'Black Panther figurine', 'Black Widow figurine', 'Captain America figurine', 'Captain Marvel figurine', 'Deku figurine', 'Doctor Strange figurine', 'Eren figurine', 'Goku figurine', 'Hulk figurine', 'Ichigo figurine', 'Iron Man figurine', 'Luffy figurine', 'Naruto figurine', 'Natsu figurine', 'Saitama figurine', 'Sasuke figurine', 'Spider-Man figurine

This Anime image classification was created by Vignesh SS for Databyte Inductions. Further changes should be made like adding a pretained head to make this model function well.